# Race conditions in parallel Numba

 - Elwin van 't Wout
 - Pontificia Universidad Católica de Chile
 - IMT3870
 - 26-8-2024

This tutorial shows a race condition in a parallel for-loop that leads to code that is not thread safe.

In [1]:
from numba import njit, prange
import numpy as np

The following code sums the elements of a vector.

In [2]:
@njit(parallel=True)
def sum_vector(x):
    length = x.shape[0]
    s = 0
    for i in prange(length):
        s += x[i]
    return s   

The expected sum of a vector with elements $0,1,2,\dots,n-1$ is $(n-1)n/2$.

In [3]:
n = int(1e5)
vec = np.arange(n)
sum_exact = int((n-1)*n/2)
print("The vector with",n,"elements sums to",sum_exact)

The vector with 100000 elements sums to 4999950000


In [4]:
sum_numba = sum_vector(vec)
print("The sum calculated by Numba is:",sum_numba)

The sum calculated by Numba is: 4999950000


Now, let us create an array with four elements, and calculate the sum of the input vector for each element of the output array. With Python broadcasting, you can add each element of the input array to the entire output array.

In [5]:
@njit(parallel=True)
def sum_vector_in_array_race_condition(x):
    length = x.shape[0]
    s = np.zeros(4, dtype=np.int_)
    for i in prange(length):
        s[:] += x[i]
    return s

In [6]:
sum_race = sum_vector_in_array_race_condition(vec)
print("The sum calculated by Numba is:",sum_race)

The sum calculated by Numba is: [1927748978 2055798753 1979814415 2010307473]


The four elements of the output array should all be the sum of the input vector, but this is not the case. The code is not thread safe! Each element is different because there is a race condition.

The following adaptation solves the race condition because it changes the data types such that Numba understands the race condition and parallelises the code correctly. Specifically, you need to create a slice reference outside the loop.

In [7]:
@njit(parallel=True)
def sum_vector_in_array_safe(x):
    length = x.shape[0]
    y = np.zeros(4, dtype=np.int_)
    s = y[:]
    for i in prange(length):
        s += x[i]
    return s

In [8]:
sum_safe = sum_vector_in_array_safe(vec)
print("The sum calculated by Numba is:",sum_safe)

The sum calculated by Numba is: [704982704 704982704 704982704 704982704]
